In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv


In [2]:
df = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv")
df #ignore the warnings

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Unnamed: 0,ID,class,20150720_N,20150602_N,20150517_N,20150501_N,20150415_N,20150330_N,20150314_N,...,20140610_N,20140525_N,20140509_N,20140423_N,20140407_N,20140322_N,20140218_N,20140202_N,20140117_N,20140101_N
0,0,1,water,637.5950,658.668,-1882.030,-1924.36,997.904,-1739.990,630.087,...,NaN,-1043.160,-1942.490,267.138,NaN,NaN,211.328,-2203.020,-1180.19,433.906
1,1,2,water,634.2400,593.705,-1625.790,-1672.32,914.198,-692.386,707.626,...,NaN,-933.934,-625.385,120.059,364.858,476.972,220.878,-2250.000,-1360.56,524.075
2,3,4,water,58.0174,-1599.160,NaN,-1052.63,NaN,-1564.630,NaN,...,-1025.880,368.622,NaN,-1227.800,304.621,NaN,369.214,-2202.120,NaN,-1343.550
3,4,5,water,72.5180,NaN,380.436,-1256.93,515.805,-1413.180,-802.942,...,-1813.950,155.624,NaN,-924.073,432.150,282.833,298.320,-2197.360,NaN,-826.727
4,7,8,water,1136.4400,NaN,NaN,1647.83,1935.800,NaN,2158.980,...,1535.000,1959.430,-279.317,-384.915,-113.406,1020.720,1660.650,-116.801,-568.05,-1357.140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,10537,10538,impervious,1207.7000,984.620,NaN,1166.25,937.478,1072.700,823.896,...,1117.740,1176.600,1044.110,NaN,369.082,465.843,362.882,979.795,NaN,433.659
7996,10538,10539,impervious,2170.3500,1419.720,1361.000,1478.71,983.911,1262.110,1422.860,...,984.634,2128.970,1379.660,NaN,762.633,485.204,446.724,771.747,1589.06,506.936
7997,10541,10542,impervious,1895.6800,1454.740,NaN,1033.56,1930.380,1057.150,1471.600,...,888.408,2093.020,1232.110,1190.830,1441.460,1170.880,1095.000,1818.650,2501.72,1247.770
7998,10542,10543,impervious,3465.7400,1283.320,413.412,4391.05,1146.820,4473.050,1614.750,...,5833.760,4047.320,4515.800,433.177,277.296,744.143,NaN,3759.710,NaN,388.346


In [3]:

# Import libraries
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Data preprocessing function
def preprocess_data(data, is_training=True, preprocessing_components=None):
    """
    Comprehensive data preprocessing function
    """
    df_clean = data.copy()
    
    # Remove unnamed columns
    unnamed_cols = [col for col in df_clean.columns if 'Unnamed' in col]
    if unnamed_cols:
        df_clean = df_clean.drop(columns=unnamed_cols)
        print(f"Dropped unnamed columns: {unnamed_cols}")
    
    # Handle missing values intelligently
    for col in df_clean.columns:
        if col in ['ID', 'class']:
            continue
            
        missing_ratio = df_clean[col].isnull().mean()
        
        if missing_ratio == 0:
            continue
        elif missing_ratio <= 0.3:
            # Use training data statistics for test data
            if is_training:
                fill_value = df_clean[col].mean()
            else:
                fill_value = preprocessing_components['fill_values'].get(col, df_clean[col].mean())
            df_clean[col] = df_clean[col].fillna(fill_value)
            
        elif missing_ratio <= 0.5:
            if is_training:
                fill_value = df_clean[col].median()
            else:
                fill_value = preprocessing_components['fill_values'].get(col, df_clean[col].median())
            df_clean[col] = df_clean[col].fillna(fill_value)
        else:
            print(f"Dropping column: {col} (missing {missing_ratio*100:.1f}%)")
            df_clean = df_clean.drop(columns=[col])
    
    return df_clean

In [5]:
# Preprocess training data
print("\nPreprocessing training data...")
df_processed = preprocess_data(df, is_training=True)


Preprocessing training data...
Dropped unnamed columns: ['Unnamed: 0']


In [6]:
# Create fill values dictionary for test data consistency
fill_values = {}
for col in df_processed.columns:
    if col not in ['ID', 'class']:
        missing_ratio = df_processed[col].isnull().mean()
        if missing_ratio > 0:
            if missing_ratio <= 0.3:
                fill_values[col] = df_processed[col].mean()
            else:
                fill_values[col] = df_processed[col].median()

print(f"Processed training data shape: {df_processed.shape}")
print(f"Class distribution:\n{df_processed['class'].value_counts()}")

Processed training data shape: (8000, 29)
Class distribution:
class
forest        6159
farm           841
impervious     669
grass          196
water          105
orchard         30
Name: count, dtype: int64


In [7]:
# Encode class labels
label_encoder = LabelEncoder()
df_processed['class'] = label_encoder.fit_transform(df_processed['class'])
print(f"\nClasses: {label_encoder.classes_}")

# Separate features and target
X = df_processed.drop(columns=['ID', 'class'])
y = df_processed['class']

print(f"\nOriginal feature matrix shape: {X.shape}")


Classes: ['farm' 'forest' 'grass' 'impervious' 'orchard' 'water']

Original feature matrix shape: (8000, 27)


In [8]:
# Feature engineering: Remove highly correlated features
print("\nRemoving highly correlated features...")
correlation_matrix = X.corr().abs()
upper_triangle = correlation_matrix.where(
    np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
)
high_corr_features = [column for column in upper_triangle.columns 
                     if any(upper_triangle[column] > 0.95)]
print(f"Removing {len(high_corr_features)} highly correlated features")
X = X.drop(columns=high_corr_features)


Removing highly correlated features...
Removing 0 highly correlated features


In [9]:
# Feature selection for Logistic Regression
print("\nSelecting most important features...")
selector = SelectKBest(score_func=f_classif, k=min(25, X.shape[1]))  # Select top 25 features for LogReg
X_selected = selector.fit_transform(X, y)
selected_features = X.columns[selector.get_support()]
print(f"Selected {len(selected_features)} most important features")

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(
    X_selected, y, test_size=0.2, random_state=42, stratify=y
)


Selecting most important features...
Selected 25 most important features


In [10]:
# Scale features - StandardScaler works well with Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [11]:
# Initial Logistic Regression model
print("\n" + "="*60)
print("TRAINING LOGISTIC REGRESSION MODEL")
print("="*60)

# Base model
base_model = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=2000,
    class_weight='balanced',
    random_state=42
)


TRAINING LOGISTIC REGRESSION MODEL


In [12]:
# Cross-validation
print("Performing cross-validation...")
cv_scores = cross_val_score(base_model, X_train_scaled, y_train, cv=5, scoring='accuracy')
print(f"CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

Performing cross-validation...
CV Score: 0.6577 (+/- 0.0142)


In [13]:
# Fit base model
base_model.fit(X_train_scaled, y_train)
y_pred_base = base_model.predict(X_val_scaled)
base_accuracy = accuracy_score(y_val, y_pred_base)
print(f"Base model validation accuracy: {base_accuracy:.4f}")

Base model validation accuracy: 0.6675


In [14]:
# Hyperparameter tuning for Logistic Regression
print(f"\nHyperparameter tuning for Logistic Regression...")

param_grid = {
    'C': [0.01, 0.05, 0.1, 0.5, 1.0, 5.0, 10.0],  # Regularization strength
    'solver': ['lbfgs', 'liblinear', 'saga'],       # Different solvers
    'max_iter': [1500, 2000, 3000]                 # Ensure convergence
}

tuned_model = LogisticRegression(
    multi_class='multinomial',
    class_weight='balanced',
    random_state=42
)


Hyperparameter tuning for Logistic Regression...


In [15]:
grid_search = GridSearchCV(
    tuned_model, 
    param_grid, 
    cv=5,  # 5-fold CV for better reliability
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

print("Performing grid search...")
grid_search.fit(X_train_scaled, y_train)
final_model = grid_search.best_estimator_

Performing grid search...
Fitting 5 folds for each of 63 candidates, totalling 315 fits


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

In [16]:
# Final validation
y_pred_final = final_model.predict(X_val_scaled)
final_accuracy = accuracy_score(y_val, y_pred_final)

print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best CV score: {grid_search.best_score_:.4f}")
print(f"Final tuned model accuracy: {final_accuracy:.4f}")


Best parameters: {'C': 5.0, 'max_iter': 1500, 'solver': 'lbfgs'}
Best CV score: 0.6578
Final tuned model accuracy: 0.6675


In [17]:
# Detailed classification report
target_names = [str(c) for c in label_encoder.classes_]
print(f"\nDetailed Classification Report:")
print(classification_report(y_val, y_pred_final, target_names=target_names))


Detailed Classification Report:
              precision    recall  f1-score   support

        farm       0.43      0.61      0.50       168
      forest       0.96      0.67      0.79      1232
       grass       0.15      0.67      0.25        39
  impervious       0.69      0.75      0.72       134
     orchard       0.03      0.67      0.06         6
       water       0.23      0.67      0.34        21

    accuracy                           0.67      1600
   macro avg       0.42      0.67      0.44      1600
weighted avg       0.85      0.67      0.73      1600



In [18]:
# Load and preprocess test data
print(f"\n" + "="*60)
print("PROCESSING TEST DATA")
print("="*60)


PROCESSING TEST DATA


In [19]:
test_data = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")
print(f"Test data shape: {test_data.shape}")

Test data shape: (2845, 29)


In [20]:
# Store test IDs
test_ids = test_data['ID'].copy()

In [21]:
# Preprocess test data using same logic as training
preprocessing_components = {'fill_values': fill_values}
test_processed = preprocess_data(test_data, is_training=False, 
                                preprocessing_components=preprocessing_components)

Dropped unnamed columns: ['Unnamed: 0']


In [22]:
# Apply same transformations as training data
test_features = test_processed.drop(columns=['ID'])

# Remove high correlation features (same as training)
test_features = test_features.drop(columns=high_corr_features, errors='ignore')

In [23]:
# Ensure same columns as training
missing_cols = set(selected_features) - set(test_features.columns)
for col in missing_cols:
    test_features[col] = 0  # Add missing columns with 0

In [24]:
# Add extra columns that might exist in test but not selected
extra_cols = set(test_features.columns) - set(selected_features)
if extra_cols:
    test_features = test_features.drop(columns=list(extra_cols))

In [25]:
# Select same features as training
test_features = test_features[selected_features]

# Scale test data using the same scaler
test_scaled = scaler.transform(test_features)

In [26]:
# Make predictions
print("\nMaking predictions...")
predictions = final_model.predict(test_scaled)
prediction_probabilities = final_model.predict_proba(test_scaled)

# Convert back to original class labels
predictions_original = label_encoder.inverse_transform(predictions)

# Create submission
result = pd.DataFrame({
    'ID': test_ids,
    'class': predictions_original
})


Making predictions...


In [27]:
print(f"\nPrediction distribution:")
print(pd.Series(predictions_original).value_counts())


Prediction distribution:
forest        922
farm          620
grass         449
impervious    416
orchard       241
water         197
Name: count, dtype: int64


In [28]:
# Show prediction confidence
max_probabilities = np.max(prediction_probabilities, axis=1)
print(f"\nPrediction confidence statistics:")
print(f"Mean confidence: {max_probabilities.mean():.4f}")
print(f"Min confidence: {max_probabilities.min():.4f}")
print(f"Max confidence: {max_probabilities.max():.4f}")


Prediction confidence statistics:
Mean confidence: 0.7221
Min confidence: 0.2291
Max confidence: 0.9990


In [29]:
# Save submission
result.to_csv("improved_submission.csv", index=False)
print(f"\nSubmission saved to 'improved_submission.csv'")
print(f"Expected accuracy improvement: {final_accuracy:.4f} vs your previous 0.11")


Submission saved to 'improved_submission.csv'
Expected accuracy improvement: 0.6675 vs your previous 0.11


In [30]:
# Model insights
print(f"\n" + "="*60)
print("MODEL INSIGHTS")
print("="*60)
print(f"Selected features: {list(selected_features)}")
print(f"Final regularization strength (C): {final_model.C}")
print(f"Solver used: {final_model.solver}")
print(f"Number of iterations: {final_model.n_iter_}")

# Show coefficients for top features if not too many classes
if len(label_encoder.classes_) <= 5:
    print(f"\nTop feature coefficients by class:")
    coef_df = pd.DataFrame(
        final_model.coef_.T,
        columns=[f'Class_{cls}' for cls in label_encoder.classes_],
        index=selected_features
    )
    # Show top 5 most important features by absolute coefficient sum
    feature_importance = coef_df.abs().sum(axis=1).sort_values(ascending=False)
    print(coef_df.loc[feature_importance.head().index])


MODEL INSIGHTS
Selected features: ['20150720_N', '20150602_N', '20150517_N', '20150501_N', '20150415_N', '20150330_N', '20150314_N', '20150226_N', '20150210_N', '20150125_N', '20150109_N', '20141117_N', '20141016_N', '20140813_N', '20140626_N', '20140610_N', '20140525_N', '20140509_N', '20140423_N', '20140407_N', '20140322_N', '20140218_N', '20140202_N', '20140117_N', '20140101_N']
Final regularization strength (C): 5.0
Solver used: lbfgs
Number of iterations: [58]
